In [ ]:

import pickle
from torchvision import transforms
import sys
import os
sys.path.append(os.path.abspath(".."))
from models import model_dict
from utils import NormalizeByChannelMeanStd
import numpy as np
from torchvision.datasets import CIFAR10, CIFAR100, ImageFolder
from dataset import prepare_train_test_dataset
from torch.utils.data import DataLoader, Dataset, Subset
import torch
import pickle
from itertools import cycle
from utils.evaluation import Hook_handle, analysis, get_micro_eval, get_acc, get_micro_eval_seperate_correct
import pandas as pd
import argparse
import random
import copy
from types import SimpleNamespace
import seaborn as sns
import matplotlib.pyplot as plt
dataset = ['cifar10', 'cifar100', 'TinyImagenet', 'TinyImagenet']
architecture = ['resnet18', 'resnet50', 'resnet18', 'vgg16_bn']

backdata = {}
for arch, data in zip(architecture, dataset):
    for us in range(3):
        with open(f"assets/feature_topo_analysis/{data}_{arch}_unlearn_seed_{us}_feature_label.pkl", "rb") as f:
            seed_dict = pickle.load(f)
            backdata[(data, arch, us)] = seed_dict

            # features = seed_dict[0]['retrain_feature'][target]
            # labels = seed_dict[0]['retrain_label'][target]
            # weights = seed_dict[0]['retrain_weight'][target]
            # bias = seed_dict[0]['retrain_bias'][target]
            # print((torch.max(features @ weights.T + bias, dim=1).indices == labels).sum() / len(torch.max(features @ weights.T + bias, dim=1).indices == labels))

In [ ]:
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA

target = 'forget'
# init_seed = 0
# arch, data = 'resnet18', 'cifar10'
# us = 0

for arch, data, NUM_CLASS_VISIBLE, alpha, perplexity in zip(architecture, dataset, [3, 6, 12, 12], [0.5, 0.5, 0.5, 0.5], [30, 10, 5, 5]):
    for us in range(3): # 3
       for init_seed in range(3): # 3 
            confidences = backdata[(data, arch, us)][init_seed]['orig_confidence'][target]
            weights = backdata[(data, arch, us)][init_seed][f'orig_weight'][target]
            # confidences = backdata[(data, arch, 0)][init_seed]['retrain_confidence'][target]
            # NUM_CLASS_VISIBLE = 3
            target_labels = np.random.choice(np.arange(weights.shape[0]), size=NUM_CLASS_VISIBLE, replace=False).astype(int)
            target_labels.sort()
            target_labels
            palette = sns.color_palette("tab10", n_colors=weights.shape[0])


            for mode in ['orig', 'retrain']:
                features = backdata[(data, arch, 0)][init_seed][f'{mode}_feature'][target]
                labels = backdata[(data, arch, 0)][init_seed][f'{mode}_label'][target]
                weights = backdata[(data, arch, 0)][init_seed][f'{mode}_weight'][target]
                bias = backdata[(data, arch, 0)][init_seed][f'{mode}_bias'][target]
                print((torch.max(features @ weights.T + bias, dim=1).indices == labels).sum() / len(torch.max(features @ weights.T + bias, dim=1).indices == labels))
                
                features_np = features.numpy()
                labels_np = labels.numpy()
                confidences_np = confidences.numpy()
                weights_np = weights.cpu().numpy()
                bias_np = bias.cpu().numpy()
                conf_norm = np.argsort(np.argsort(confidences_np)) / (len(confidences_np) - 1) 


                mask = np.isin(labels_np, target_labels)
                features_np = features_np[mask]
                labels_np = labels_np[mask]
                confidences_np = confidences_np[mask]
                weights_np = weights_np[target_labels]
                bias_np = bias_np[target_labels]
                conf_norm = conf_norm[mask]
                

                # tsne = TSNE(n_components=2, random_state=42, perplexity=50)
                tsne = TSNE(n_components=2, perplexity=perplexity, metric='cosine') #  
                features_np = features_np / np.linalg.norm(features_np, axis=1, keepdims=True)
                weights_np = weights_np / np.linalg.norm(weights_np, axis=1, keepdims=True)
                tsne_all_input = np.concatenate([features_np, weights_np], axis=0) # , np.zeros((1, weights_np.shape[1]))
                tsne_all_input = tsne_all_input @ weights_np.T
                tsne_all_output = tsne.fit_transform(tsne_all_input)

                features_tsne = tsne_all_output[:features_np.shape[0]]
                weight_tsne = tsne_all_output[features_np.shape[0]:]
                # zero_point = tsne_all_output[-1]

                import matplotlib.colors as mcolors
                for qwer, title in enumerate(['low_conf', 'high_conf']):
                    plt.figure(figsize=(3, 3))
                    # plt.scatter(zero_point[0], zero_point[1],
                    #                 color='k', edgecolors='k', marker='o', s=100, linewidths=1.5)

                    white = np.array([1.0, 1.0, 1.0])
                    custom_legend = []
                    for i, label in enumerate(target_labels): #range(weights.shape[0]):

                        idx = labels_np == label
                        low_conf_mask = (conf_norm < alpha) & idx  
                        high_conf_mask = (~low_conf_mask) & idx

                        base_color = np.array(palette[int(i)])

                        # alphas = 1.0 - conf_norm[idx] ** 2 # 
                        # colors = [tuple(alpha * base_color + (1 - alpha) * white) for alpha in alphas]
                        plt.scatter(weight_tsne[i, 0], weight_tsne[i, 1],
                                    color=base_color*0.6, edgecolors='k', marker='D', s=50, linewidths=1.5,
                                    label=f'Weight{int(label)}', zorder=1)

                        # plt.scatter(features_tsne[idx, 0], features_tsne[idx, 1],
                        #             color=colors, s=40, label=f'Label {int(label)}', edgecolors='none')

                        zorders_low = np.random.uniform(1.5, 2.5, size=features_tsne[low_conf_mask].shape[0])
                        zorders_high = np.random.uniform(1.5, 2.5, size=features_tsne[high_conf_mask].shape[0])


               
                        alphas_low = 1.0 - conf_norm[low_conf_mask] ** 2 # 
                        # colors_low = [tuple(alpha * base_color + (1 - alpha) * white) for alpha in alphas_low]
                        colors_low = base_color
                        alphas_high = 1.0 -  conf_norm[high_conf_mask] ** 2 # 
                        # colors_high = [tuple(alpha * base_color + (1 - alpha) * white) for alpha in alphas_high]
                        colors_high = base_color
                        
                        if qwer == 0: # Low confidence points
                            for pt, z in zip(features_tsne[low_conf_mask], zorders_low):
                                plt.scatter(pt[0], pt[1],
                                            color=base_color, s=60, edgecolors='none',
                                            marker='o', zorder=z, alpha=0.5, label=f'class{int(label)}')
                        elif qwer == 1: # High confidence points
                            for pt, z in zip(features_tsne[high_conf_mask], zorders_high):
                                plt.scatter(pt[0], pt[1],
                                            color=base_color, s=60, edgecolors='none',
                                            marker='o', zorder=z, alpha=0.5, label=f'class{int(label)}')
                                            
                 
                    # Weight 
                    import matplotlib.lines as mlines
                    import matplotlib.patches as mpatches
                    weight_patch = mlines.Line2D([], [], color='k',
                                                marker='D', linestyle='None',
                                                markeredgecolor='k', markersize=6, label=f'Weight')
                    
                    # Low confidence 
                    circ_patch = mlines.Line2D([], [], color='k', alpha=0.5,
                                            marker='o', linestyle='None',
                                            markersize=6, label=f'Feature')
                    custom_legend.extend([weight_patch, circ_patch])
                    plt.legend(
                        handles=custom_legend,
                        loc='upper center',
                        bbox_to_anchor=(0.5, 1.12),  
                        fontsize=8,
                        ncol=3,
                        frameon=True,
                        handletextpad=0.5,  
                        columnspacing=1.6,  
                        handlelength=1.2    
                    )
                    # plt.title("t-SNE of Original Features\n(Confidence → Saturation)")
                    # plt.xlabel("Component 1")
                    # plt.ylabel("Component 2")
                    plt.xticks([])
                    plt.yticks([])
                    plt.tight_layout()
                    plt.grid(True)

                    plt.savefig(f'assets/feature_topo_analysis_figure/{data}_{arch}_unlearn_seed_{us}_init_seed_{init_seed}_tsne_{mode}_{title}.pdf', bbox_inches='tight', format='pdf')
                    print(mode, title)
                    plt.show()
                    plt.close()